In [1]:
#!/usr/bin/python3
# -*- coding: utf-8 -*-
from PyQt5 import Qt
import pyqtgraph as pg
import numpy as np
import csv
import os as dirinfo

# создание папок
# исходные файлы вкладываются пользоватлем в папку ./data перед запуском скрипта
if not dirinfo.path.exists('./data'):
    dirinfo.mkdir('./data')
if not dirinfo.path.exists('./resault'):
    dirinfo.mkdir('./resault')

In [2]:
#чтение основного файла и выделение нужных столбцов
def fileReader(text):
    with open('./data/' + text) as f:
        reader = np.array(list(csv.reader(f)))
    return np.transpose(reader[1::])[2:14:5]

In [4]:
#количество каждого трека в файле
def countsTemp(tempTrack):
    counts = [i for i in range(tempTrack[-1]+1)]
    
    for i in counts:
        counts[i] = list(tempTrack).count(i)    
    return counts

In [5]:
#разбиение по трекам и запись нового файла (номер трека, время в кадрах)
def newFileCreater(tempTrack, tempTime, tempD, text):
            
    counts = countsTemp(tempTrack)
        
    tMax = sorted(tempTime, reverse = True)[0]
    
    summ = 0
    check = 0
    paternTrack = np.array(range(tMax+1), dtype = '<U32')
    
    for i in range(tMax+1):
        paternTrack[i] = ''
    t = np.array(range(0), dtype = '<U32')
    
    for i in counts:
        tempData = paternTrack.copy()
        for j in range(summ, summ+i):
            tempData[tempTime[j]] = tempD[j]
        t = np.insert(t, check, tempData)
        check+=len(tempData)
        summ+=i
        
    t = np.insert(t, 0, list(range(tMax+1)))
    t = np.transpose(t.reshape(len(counts)+1, tMax+1))
    tempHead = np.array(["Track " + str(i) for i in range(len(counts))])
    tempHead = np.insert(tempHead, 0, "Time")
    with open('./resault/'+ 'New'+ text, 'w') as f:
        writer = csv.writer(f)
        writer.writerow(tempHead)
        for row in t:
            writer.writerow(row)
    

In [6]:
#UI
class Window(Qt.QWidget):
    count = 0
    tempTrack =0
    tempTime = 0
    tempD = 0
    nameFile = ''
    temp = 0
    
    def __init__(self):
        super().__init__()
        self.initUI()
    
    def initUI(self):
        #объявление слоев, и сеток
        grid = Qt.QGridLayout()
        layoutBtn = Qt.QVBoxLayout()
        self.setLayout(grid)
        layoutFlash = Qt.QHBoxLayout()
        layoutTable = Qt.QHBoxLayout()
        layoutParametrs = Qt.QVBoxLayout()
        layoutErrows = Qt.QVBoxLayout()
        layoutYandLenth = Qt.QVBoxLayout()
        
        #кнопок, выпадающих меню и другого интерфейса
        self.btnFile = Qt.QPushButton("Load File")
        self.btnFile.clicked.connect(self.load_File)
        
        self.btnNewFile1 = Qt.QPushButton("Save Resault")
        self.btnNewFile1.clicked.connect(self.create_New_File)
        
        self.btnNewFile2 = Qt.QPushButton("Save Fitting")
        #self.btnNewFile2.clicked.connect(self.create_New_File)
        
        self.btnNewFile3 = Qt.QPushButton("Save Flash")
       # self.btnNewFile3.clicked.connect(self.create_New_File)
        
        self.fdilCombo = Qt.QComboBox()
        self.fdilCombo.addItems(dirinfo.listdir('./data'))
        
        self.btnPlot = Qt.QPushButton("Plot")
        self.btnPlot.clicked.connect(self.view_plot)
        self.combo = Qt.QComboBox()
        
        self.view = pg.PlotWidget()
        
        self.btn = Qt.QPushButton("clear")
        self.btn.clicked.connect(self.clear_plot)
        
        self.btnFlash = Qt.QPushButton("flash")
        #self.btnFlash.clicked.connect(self.flash_resault)        
        self.lineTextFlash = Qt.QLineEdit()       
        layoutFlash.addWidget(self.lineTextFlash)        
        layoutFlash.addWidget(self.btnFlash)

        self.titleParametrs = Qt.QLabel("Parametrs")
        self.titleErrows = Qt.QLabel("Errows")
        
        
        self.parametrsTable = pg.TableWidget()
        self.errowTable = pg.TableWidget()
        layoutParametrs.addWidget(self.titleParametrs)
        layoutParametrs.addWidget(self.parametrsTable)
        layoutErrows.addWidget(self.titleErrows)
        layoutErrows.addWidget(self.errowTable)
        layoutTable.addLayout(layoutParametrs)
        layoutTable.addLayout(layoutErrows)
        
        
        self.titleY = Qt.QLabel("Y")
        self.lineY = Qt.QLineEdit()
        self.titleLenth = Qt.QLabel("Lenth")
        self.lineLenth = Qt.QLineEdit()
        layoutYandLenth.addWidget(self.titleY)
        layoutYandLenth.addWidget(self.lineY)
        layoutYandLenth.addWidget(self.titleLenth)
        layoutYandLenth.addWidget(self.lineLenth)
        
        #Разметка UI приложения
 ################  ###################  ##############

        grid.addWidget(self.btn, 0, 0)
        grid.addWidget(self.fdilCombo, 0, 1)
        
        grid.addLayout(layoutBtn, 1, 1)
        layoutBtn.addWidget(self.btnFile)        
        layoutBtn.addWidget(self.combo)
        layoutBtn.addWidget(self.btnPlot)
        layoutBtn.addLayout(layoutFlash)
        
        layoutBtn.addStretch(1)

        #layoutV.addWidget(Qt.QLabel("The Best Proga"))
        grid.addWidget(self.view, 1, 0)
        layoutBtn.addWidget(self.btnNewFile1)
        layoutBtn.addWidget(self.btnNewFile2)
        layoutBtn.addWidget(self.btnNewFile3)
        grid.addLayout(layoutTable, 2, 0)
        grid.addLayout(layoutYandLenth, 2, 1)
        
        
        
    
    def create_New_File(self):
        newFileCreater(self.tempTrack, self.tempTime, self.tempD, self.nameFile)
    
    #Загрузка данных из файла
    def load_File(self): 
        self.nameFile = self.fdilCombo.currentText()
        self.temp = fileReader(self.nameFile)
        self.tempTrack = np.array([int(i) for i in self.temp[0]])
        self.tempTime = np.array([int(i) for i in self.temp[1]])
        self.tempD = np.array([float(i) for i in self.temp[2]])
        self.count = countsTemp(self.tempTrack)
        self.combo.clear()
        self.combo.addItems(np.array([str(i) for i in range(len(self.count))]))
        
    #Отрисовка графика трека
    def view_plot(self):
        
        self.view.clear()
        numberTrack = self.combo.currentIndex()
        a = 0
        b = 0
        for i in range(0, numberTrack+1):
            a = b
            b+=self.count[i]
        self.view.plot(self.tempTime[a:b:1], self.tempD[a:b:1], symbol = 'o')
        self.view.showGrid(True, True, 1.0)
        self.view.getViewBox().autoRange()
        
    def clear_plot(self):
        self.view.clear()


if __name__ == "__main__":
    app = Qt.QApplication([])
    w = Window()
    w.show()
    app.exec()